In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.decomposition import PCA

In [2]:
cd ml-1m

/Users/apple/Desktop/zyl/SJSU/257/lab2/ml-1m


In [3]:
ratings_df = pd.read_csv("ratings.dat", sep="::", names = ['user', 'movie', 'rating', 'timestamp'])
movies_df = pd.read_csv("movies.dat", sep="::", names = ['movie', 'Title', 'Genres'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
movies_df.head()

,movie,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_df.head()

,user,movie,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
R_df = ratings_df.pivot(index = 'user', columns ='movie', values = 'rating').fillna(0)
R_df.head()

movie,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [8]:
U, sigma, Vt = svds(R_demeaned, k = 50)

In [9]:
sigma = np.diag(sigma)

In [10]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [11]:
def recommend_movies(predictions_df, user, movies_df, original_ratings_df, num_recommendations=5):
    
    user_row_number = user - 1
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    user_data = original_ratings_df[original_ratings_df.user == (user)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movie', right_on = 'movie').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(user, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    recommendations = (movies_df[~movies_df['movie'].isin(user_full['movie'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie',
               right_on = 'movie').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [12]:
already_rated, predictions = recommend_movies(preds_df, 100, movies_df, ratings_df, 10)

User 100 has already rated 76 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [13]:
already_rated.head(10)

,user,movie,rating,timestamp,Title,Genres
1,100,800,5,977593915,Lone Star (1996),Drama|Mystery
63,100,527,5,977594839,Schindler's List (1993),Drama|War
16,100,919,5,977594947,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
17,100,924,4,977594873,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
29,100,969,4,977594044,"African Queen, The (1951)",Action|Adventure|Romance|War
22,100,2406,4,977594142,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
47,100,318,4,977594839,"Shawshank Redemption, The (1994)",Drama
20,100,858,4,977593950,"Godfather, The (1972)",Action|Crime|Drama
49,100,329,4,977594297,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
50,100,260,4,977593595,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi


In [14]:
R

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [15]:
normalised_mat = R - np.matrix(np.mean(R, 1)).T

In [16]:
cov_mat = np.cov(normalised_mat)

In [17]:
evals, evecs = np.linalg.eig(cov_mat)

In [18]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

In [19]:
def print_similar_movies(movie_data, movie, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movies_df[movie_data.movie == movie].Title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie == id].Title.values[0])

In [24]:
k = 50
movie = 1
top_n = 10

sliced = Vt.T[:, :k]
top_indexes = top_cosine_similarity(sliced, movie, top_n)
print_similar_movies(movies_df, movie, top_indexes)


Recommendations for Toy Story (1995): 

Toy Story (1995)
Gulliver's Travels (1939)
Babe (1995)
Attack of the Killer Tomatoes! (1980)
Saltmen of Tibet, The (1997)
Little Rascals, The (1994)
See the Sea (Regarde la mer) (1997)
Road to Wellville, The (1994)
Tinseltown (1998)
Firestorm (1998)
